In [ ]:
from datasets import load_dataset, Dataset
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel, get_scheduler
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
from torch.cuda.amp import autocast, GradScaler
torch.backends.cuda.matmul.allow_tf32 = True
import bitsandbytes as bnb

# Parameters
num_epochs = 20
lr = 5e-5
batch_size = 14
warmup_steps= 750
save_loc1 = '/home/arjun/Documents/ModelSaves/GPT2Alpaca'
save_loc2 = ''
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

In [ ]:
dataset = load_dataset("tatsu-lab/alpaca")
dataset = dataset['train']

# Making dataset smaller for fast training
dataset = dataset.select(range(52000))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(save_loc1)
model = GPT2LMHeadModel.from_pretrained(save_loc1)

optimizer = AdamW(model.parameters(), lr=lr)

In [ ]:
new_dataset = {'input_ids': [], 'attention_mask': []}

for example in dataset:
    input_text = example['text'].replace('###', '')
    encoded_data = tokenizer('' + input_text + '', truncation=True, max_length=768, padding="max_length")
    new_dataset['input_ids'].append(encoded_data['input_ids'])
    new_dataset['attention_mask'].append(encoded_data['attention_mask'])

new_dataset = Dataset.from_dict(new_dataset)
new_dataset.set_format("torch")

# DataLoader
dataloader = DataLoader(new_dataset, shuffle=True, batch_size=batch_size, pin_memory=True)

In [ ]:
# Optimizer and scheduler
num_training_steps = num_epochs * len(dataloader)
lr_scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
model.resize_token_embeddings(len(tokenizer))
model.to(device)
model = torch.compile(model)
scaler = GradScaler()

In [1]:
# Training loop
progress_bar = tqdm(range(num_training_steps-1), desc='Training', unit='steps')
model.train()
ep = 0

for epoch in range(num_epochs):
    total_train_loss = 0
    for batch in dataloader:
        batch_data = batch['input_ids'].to(device)
        attention = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        with autocast():
            outputs = model(batch_data,
                            labels=batch_data,
                            attention_mask=attention,
                            token_type_ids=None
                            )

            loss = outputs[0]
            batch_loss = loss.item()
            total_train_loss += batch_loss

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        lr_scheduler.step()
        progress_bar.update(1)

    avg_train_loss = total_train_loss / len(dataloader)
    ep += 1
    print('Epoch:', ep, 'Average training loss =', avg_train_loss)
    total_train_loss = 0 




===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
ERROR: /home/arjun/NewPytorchEnv/bin/python3.10: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizer

/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/cuda-11.7/lib64')}
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/cuda-11.7/lib64: did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/home/arjun/NewPytorchEnv/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were

cuda


Using custom data configuration tatsu-lab--alpaca-2b32f0433506ef5f
Found cached dataset parquet (/home/arjun/.cache/huggingface/datasets/tatsu-lab___parquet/tatsu-lab--alpaca-2b32f0433506ef5f/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

Training:   0%|          | 0/37149 [00:00<?, ?steps/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

[2023-07-29 21:46:36,855] torch._inductor.utils: [WARNING] using triton random, expect difference from eager


KeyboardInterrupt: 

In [ ]:
question = 'Give me tips to be healthy'

prompt = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
 Instruction:{question}
 Response: """
generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

sample_outputs = model.generate(
                                generated, 
                                do_sample=True,   
                                top_k=100,
                                max_length=100,
                                max_new_tokens=200,
                                top_p=.95, 
                                num_return_sequences= 5,
                                temperature = .9,
                                )

for i, sample_output in enumerate(sample_outputs):
    ans = tokenizer.decode(sample_output, skip_special_tokens=True).split('Response: ')
    print("\n\n-------------------------------------------------------------------------------------------------------------------------------------------")
    try:        print(f'<-{i+1}-> {ans[1]}')
    except:
        print(f'<-{i+1}-> ___No response___')